<a href="https://colab.research.google.com/github/UrmilaAirsang/Pytorch/blob/main/MNIST_IncreaseOrder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
#  Try  
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 15, 3, padding=0) #input -? OUtput? RF
        self.BN1 = nn.BatchNorm2d(15)
        self.conv2 = nn.Conv2d(15, 20, 3, padding=0)
        self.BN2 = nn.BatchNorm2d(20)        
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(20, 32, 3, padding=0)   
        self.BN3 = nn.BatchNorm2d(32)  
        self.conv4 = nn.Conv2d(32, 20, 3, padding=0)   
        self.BN4 = nn.BatchNorm2d(20)
        self.pool2 = nn.MaxPool2d(2, 2)        
        self.conv5 = nn.Conv2d(20, 15, 1)
        self.BN5 = nn.BatchNorm2d(15)        
         
        #self.pool2 = nn.MaxPool2d(2, 2)
        self.conv6 = nn.Conv2d(15, 10, 1)
        self.ap1 = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.pool1(self.BN2(F.relu(self.conv2(self.BN1(F.relu(self.conv1(x)))))))
        x = self.pool2(self.BN4(F.relu(self.conv4(self.BN3(F.relu(self.conv3(x)))))))
        x = self.ap1(self.conv6(self.BN5(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 15, 26, 26]             150
       BatchNorm2d-2           [-1, 15, 26, 26]              30
            Conv2d-3           [-1, 20, 24, 24]           2,720
       BatchNorm2d-4           [-1, 20, 24, 24]              40
         MaxPool2d-5           [-1, 20, 12, 12]               0
            Conv2d-6           [-1, 32, 10, 10]           5,792
       BatchNorm2d-7           [-1, 32, 10, 10]              64
            Conv2d-8             [-1, 20, 8, 8]           5,780
       BatchNorm2d-9             [-1, 20, 8, 8]              40
        MaxPool2d-10             [-1, 20, 4, 4]               0
           Conv2d-11             [-1, 15, 4, 4]             315
      BatchNorm2d-12             [-1, 15, 4, 4]              30
           Conv2d-13             [-1, 10, 4, 4]             160
        AvgPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:
#image augmentation: A
torch.manual_seed(1)
batch_size = 64

customTransforms =transforms.Compose([
                        #transforms.ToPILImage(),
                        #transforms.ColorJitter(brightness=0.5),
                        #transforms.RandomGrayscale(p=0.2),                                                                           
                        transforms.RandomAffine(degrees=15,translate = (0.1, 0.1), scale = (0.9, 1.1)),
                        #transforms.RandomRotation(15),                        
                        #transforms.GaussianBlur(5),                                                  
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=customTransforms),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.1,patience=5, verbose= 'true')
# gamma = decaying factor
scheduler = StepLR(optimizer, step_size=10, gamma=0.2)

#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(model.parameters(),lr=0.01)
for epoch in range(1, 20):
    losses=[]
    print('Epoch:', epoch)#,'LR:',scheduler.get_lr()
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    #scheduler.step()
    #scheduler.get_last_lr()

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.06413646042346954 batch_id=937: 100%|██████████| 938/938 [00:25<00:00, 36.46it/s]
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0771, Accuracy: 9784/10000 (97.8400%)

Epoch: 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0196, Accuracy: 9933/10000 (99.3300%)

Epoch: 18


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0235, Accuracy: 9934/10000 (99.3400%)

Epoch: 19


loss=0.024989385157823563 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.28it/s]



Test set: Average loss: 0.0230, Accuracy: 9921/10000 (99.2100%)

